In [1]:
#Load libraries that would be required for execution
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import Imputer, LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
%matplotlib inline

In [2]:
# Load the train and test data to dataframe
train_data = pd.read_csv("exercise_05_train.csv")
test_data = pd.read_csv("exercise_05_test.csv")

In [3]:
# Determine the number of rows and columns in the data
train_data.shape

(40000, 101)

In [4]:
#Take a glimpse of at dataframe
train_data.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x91,x92,x93,x94,x95,x96,x97,x98,x99,y
0,0.963686,6.627185,-45.224008,9.477531,-3.216532,13.216874,9.754747,5.245851,-1.102918,-2.867482,...,0.988829,0.313772,asia,1.380664,-16.388994,5.326730,4.187294,0.045549,-3.646841,0
1,-1.770062,-23.610459,-0.964003,-31.981497,-10.294599,-10.240251,-1.518888,-1.675208,0.498134,-0.614390,...,-2.162863,1.809807,asia,2.500590,4.338834,-1.583225,-1.172417,0.011216,0.097180,0
2,9.962401,-8.349849,23.248891,-24.196879,8.937480,10.965000,-7.490596,-3.025094,0.595807,0.382732,...,1.779660,9.528113,asia,1.396475,7.839188,10.402396,1.288991,0.008209,-4.132316,0
3,-5.780709,-25.261584,1.383115,-11.786929,7.993078,-11.245752,-2.607351,-3.513896,-0.614235,-1.453979,...,-0.203206,4.892248,asia,0.744317,7.380982,7.599323,-8.022884,-0.067624,-1.796198,0
4,1.211541,1.119963,7.512938,21.987312,-5.155392,10.339416,3.045180,-0.619230,-0.928068,0.405024,...,0.248724,18.694990,asia,1.703196,-11.552129,0.381768,-3.550471,-0.055180,-3.344490,0


In [9]:
# Get a description of the data
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 101 entries, x0 to y
dtypes: float64(94), int64(1), object(6)
memory usage: 30.8+ MB


In [6]:
# Get summary statistics of the dataframe
train_data.describe()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x90,x91,x92,x94,x95,x96,x97,x98,x99,y
count,39988.000000,39990.000000,39993.000000,39987.000000,39993.000000,39992.000000,39991.000000,39990.000000,39994.000000,39993.000000,...,39993.000000,39998.000000,39994.000000,39991.000000,39993.000000,39986.000000,39991.000000,39996.000000,39987.000000,40000.000000
mean,2.020255,-3.924559,1.006619,-1.378330,0.070199,-0.715213,-0.002706,-0.025689,-0.354808,-0.017024,...,-11.953656,0.002118,0.024486,-0.012012,0.019123,-0.317345,-0.562453,0.000484,0.179715,0.203675
std,9.590599,18.768656,21.062970,29.397779,20.243287,18.268807,6.789876,5.600785,1.598967,1.346529,...,132.765199,1.643530,7.222472,2.813507,8.553992,9.321339,4.050658,0.060034,4.506750,0.402735
min,-36.842503,-79.156374,-89.728356,-126.652341,-76.412886,-73.743342,-26.354840,-21.541786,-6.567005,-5.278747,...,-580.645686,-6.353465,-31.445868,-12.188410,-38.965443,-42.409405,-16.287032,-0.250606,-18.876474,0.000000
25%,-4.461433,-16.591552,-13.230956,-21.297149,-13.580632,-13.092873,-4.611958,-3.803039,-1.423225,-0.918678,...,-100.071802,-1.107204,-4.860456,-1.924613,-5.730549,-6.561087,-3.293697,-0.039977,-2.879191,0.000000
50%,2.022412,-4.061703,1.184946,-1.224625,0.091600,-0.657601,0.005821,-0.002787,-0.355350,-0.029236,...,-9.456756,-0.007369,0.011971,-0.001604,0.078915,-0.228287,-0.548699,0.000486,0.171954,0.000000
75%,8.389979,8.529110,15.221205,18.530623,13.722427,11.610239,4.519461,3.739330,0.717467,0.886102,...,78.150364,1.111249,4.881166,1.886416,5.827534,5.939217,2.138787,0.041186,3.237456,0.000000
max,44.478690,77.682652,84.625640,117.004453,85.934044,74.465465,33.866723,27.609576,8.180336,5.179831,...,520.261927,7.432107,30.457771,11.852046,35.543669,38.649613,17.069095,0.221392,18.097897,1.000000


In [7]:
#Take a glimpse of the test data
test_data.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99
0,0.519093,-4.606038,13.707586,-17.990903,12.873394,14.910935,2.915341,-10.110081,1.628317,-0.365064,...,-138.889799,-0.246395,-1.912581,asia,-0.633811,1.181750,-0.836542,-2.144871,0.010353,-4.819828
1,-12.357004,13.874141,14.052924,34.129247,34.511107,34.583336,-0.482540,-6.583407,-4.326799,-1.216928,...,117.305737,0.316266,6.838756,asia,-4.357530,0.390822,14.052604,-2.744911,-0.005648,-1.253519
2,1.834922,2.665252,-44.873210,21.941920,10.102981,5.962249,-5.733909,-4.061670,-0.172269,0.096051,...,-66.548569,-0.657101,3.128596,asia,1.880922,0.810115,15.737267,-9.031679,0.089295,-4.285350
3,20.972483,11.548506,-40.924625,-35.296796,-35.253101,-14.601890,5.045075,10.841771,-1.872260,-0.002583,...,-150.133466,-0.391609,-4.306940,asia,-2.326108,-1.968523,-4.292831,-1.674606,-0.088565,-0.981937
4,-9.916044,5.509811,31.749288,-0.803916,-4.005098,20.912490,0.419346,-2.949516,1.057176,-0.338547,...,-23.313528,-3.207335,2.294935,asia,3.654200,12.774511,-10.603162,0.971344,0.062776,1.557684


In [11]:
test_data.shape

(10000, 100)

## Data Cleaning

## Clean Train Data

In [12]:
# Determine how many columns columns contain categorical data

In [13]:
train_data.dtypes[train_data.dtypes == 'object']

x34    object
x35    object
x41    object
x45    object
x68    object
x93    object
dtype: object

In [14]:
# Take a peak at the categorical columns
train_data[["x34","x35","x41","x45","x68","x93"]].head()

,x34,x35,x41,x45,x68,x93
0,chrystler,thur,$-865.28,0.02%,sept.,asia
1,volkswagon,thur,$325.27,-0.01%,July,asia
2,bmw,thurday,$743.91,0.0%,July,asia
3,nissan,thurday,$538.48,0.01%,July,asia
4,volkswagon,wed,$-433.65,0.0%,Jun,asia


In [15]:
# Since the x41 and the x45 columns contain '$%' charactors with are unnecessry
# They will be removed and the columns converted to float

In [16]:
train_data["x41"]=[float(str(x).replace("$","")) for x in list(train_data["x41"])]
train_data["x45"]=[float(str(x).replace("%","")) for x in list(train_data["x45"])]

In [17]:
train_data[["x34","x35","x41","x45","x68","x93"]].dtypes

x34     object
x35     object
x41    float64
x45    float64
x68     object
x93     object
dtype: object

In [18]:
train_data.isna().sum()

x0     12
x1     10
x2      7
x3     13
x4      7
x5      8
x6      9
x7     10
x8      6
x9      7
x10     7
x11     9
x12    11
x13    15
x14     5
x15     8
x16     8
x17    11
x18    14
x19     8
x20     4
x21    12
x22     6
x23     9
x24    11
x25     7
x26    10
x27     7
x28     9
x29     3
       ..
x71     6
x72     9
x73    13
x74     9
x75    10
x76     8
x77    11
x78     8
x79     7
x80     7
x81     3
x82     8
x83     4
x84     3
x85    12
x86     7
x87     9
x88     5
x89    10
x90     7
x91     2
x92     6
x93     7
x94     9
x95     7
x96    14
x97     9
x98     4
x99    13
y       0
Length: 101, dtype: int64

In [19]:
# The train data contains numerous nan values. This values will be handles in
# a pipeline later

In [296]:
train_data["x34"].value_counts()

volkswagon    12622
Toyota        10968
bmw            7262
Honda          5174
tesla          2247
chrystler      1191
nissan          326
ford            160
mercades         31
chevrolet        11
Name: x34, dtype: int64

In [20]:
# All values of the x34 column are unique

In [297]:
train_data["x35"].value_counts()

wed          14820
thurday      13324
wednesday     5938
thur          4428
tuesday        884
friday         517
monday          53
fri             26
Name: x35, dtype: int64

In [21]:
# x35 contains repeated data that is entered differently,
# thurday and thur should be the same entry. Same to wednesday and friday

In [298]:
train_data["x68"].value_counts()

July       11114
Jun         9317
Aug         8170
May         4744
sept.       3504
Apr         1629
Oct          885
Mar          407
Nov          145
Feb           48
Dev           16
January       12
Name: x68, dtype: int64

In [23]:
# The x68 column contains only unique data and so is x93

In [299]:
train_data["x93"].value_counts()

asia       35384
america     3167
euorpe      1442
Name: x93, dtype: int64

In [24]:
# The cell below will be used to clean redudant columns in the dataset

In [0]:
for x,y in enumerate(train_data['x35']):
    if y == 'thur':
        train_data['x35'][x]='thursday'
        print("replaced", y ," at row ",x)
    elif y == 'thurday':
        train_data['x35'][x]='thursday'
        print("replaced", y ," at row ",x)
    elif y == 'wed':
        train_data['x35'][x]='wednesday'
        print("replaced", y ," at row ",x)
    elif y == 'fri':
        train_data['x35'][x] ='friday'
        print("replaced", y ," at row ",x)
    else:
        pass

In [26]:
train_data["x35"].value_counts()

wednesday    20758
thursday     17752
tuesday        884
friday         543
monday          53
Name: x35, dtype: int64

In [27]:
# All redudant values have been replaced

### Clean Test Data

In [28]:
# The same procedure will be repeated on the test dataset

In [301]:
test_data.dtypes[test_data.dtypes == 'object']

x34    object
x35    object
x68    object
x93    object
dtype: object

In [302]:
test_data.shape

(10000, 101)

In [0]:
Y=train_data.iloc[-1].values

In [303]:
test_data[["x34","x35","x41","x45","x68","x93"]].head()

,x34,x35,x41,x45,x68,x93
0,bmw,thursday,107.93,0.00,Jun,asia
1,tesla,thursday,-600.43,0.02,May,asia
2,Honda,thursday,103.08,-0.00,Jun,asia
3,volkswagon,thursday,1518.78,-0.01,sept.,asia
4,volkswagon,thursday,-2324.39,-0.00,Jun,asia


In [0]:
test_data["x41"]=[float(str(x).replace("$","")) for x in list(test_data["x41"])]
test_data["x45"]=[float(str(x).replace("%","")) for x in list(test_data["x45"])]

In [0]:
test_data.isna().sum()

In [306]:
test_data["x35"].value_counts()

wednesday    5196
thursday     4408
tuesday       228
friday        144
monday         21
Name: x35, dtype: int64

In [0]:
for x,y in enumerate(test_data['x35']):
    if y == 'thur':
        test_data['x35'][x]='thursday'
        print("replaced", y ," at row ",x)
    elif y == 'thurday':
        test_data['x35'][x]='thursday'
        print("replaced", y ," at row ",x)
    elif y == 'wed':
        test_data['x35'][x]='wednesday'
        print("replaced", y ," at row ",x)
    elif y == 'fri':
        test_data['x35'][x] ='friday'
        print("replaced", y ," at row ",x)
    else:
        pass

In [0]:
test_data["x35"].value_counts()

In [ ]:
# The cell below will be used to store the cleaned dataset in a csv file. The clean up 
# process can take a while if repeated

In [0]:
train_data = pd.to_csv("clean_train_data.csv")
test_data = pd.to_csv("clean_test_data.csv")

In [0]:
#train_data = pd.read_csv("clean_train_data.csv")
#test_data = pd.read_csv("clean_test_data.csv")

In [29]:
# The code before will be used to select dataframe and perform encoding of categorical data

In [0]:
class DataFrameSelector(BaseEstimator, TransformerMixin): # Select attributes of a dataframe
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
    
class CategoricalEncoder(BaseEstimator, TransformerMixin): #Encode categorical features of the dataframe 
    def __init__(self, encoding='onehot', categories='auto', dtype=np.float64,
                 handle_unknown='error'):
        self.encoding = encoding
        self.categories = categories
        self.dtype = dtype
        self.handle_unknown = handle_unknown

    def fit(self, X, y=None):

        if self.encoding not in ['onehot', 'onehot-dense', 'ordinal']:
            template = ("encoding should be either 'onehot', 'onehot-dense' "
                        "or 'ordinal', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.handle_unknown not in ['error', 'ignore']:
            template = ("handle_unknown should be either 'error' or "
                        "'ignore', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.encoding == 'ordinal' and self.handle_unknown == 'ignore':
            raise ValueError("handle_unknown='ignore' is not supported for"
                             " encoding='ordinal'")

        X = check_array(X, dtype=np.object, accept_sparse='csc', copy=True)
        n_samples, n_features = X.shape

        self._label_encoders_ = [LabelEncoder() for _ in range(n_features)]

        for i in range(n_features):
            le = self._label_encoders_[i]
            Xi = X[:, i]
            if self.categories == 'auto':
                le.fit(Xi)
            else:
                valid_mask = np.in1d(Xi, self.categories[i])
                if not np.all(valid_mask):
                    if self.handle_unknown == 'error':
                        diff = np.unique(Xi[~valid_mask])
                        msg = ("Found unknown categories {0} in column {1}"
                               " during fit".format(diff, i))
                        raise ValueError(msg)
                le.classes_ = np.array(np.sort(self.categories[i]))

        self.categories_ = [le.classes_ for le in self._label_encoders_]

        return self

    def transform(self, X):
        X = check_array(X, accept_sparse='csc', dtype=np.object, copy=True)
        n_samples, n_features = X.shape
        X_int = np.zeros_like(X, dtype=np.int)
        X_mask = np.ones_like(X, dtype=np.bool)

        for i in range(n_features):
            valid_mask = np.in1d(X[:, i], self.categories_[i])

            if not np.all(valid_mask):
                if self.handle_unknown == 'error':
                    diff = np.unique(X[~valid_mask, i])
                    msg = ("Found unknown categories {0} in column {1}"
                           " during transform".format(diff, i))
                    raise ValueError(msg)
                else:
                    # Set the problematic rows to an acceptable value and
                    # continue `The rows are marked `X_mask` and will be
                    # removed later.
                    X_mask[:, i] = valid_mask
                    X[:, i][~valid_mask] = self.categories_[i][0]
            X_int[:, i] = self._label_encoders_[i].transform(X[:, i])

        if self.encoding == 'ordinal':
            return X_int.astype(self.dtype, copy=False)

        mask = X_mask.ravel()
        n_values = [cats.shape[0] for cats in self.categories_]
        n_values = np.array([0] + n_values)
        indices = np.cumsum(n_values)

        column_indices = (X_int + indices[:-1]).ravel()[mask]
        row_indices = np.repeat(np.arange(n_samples, dtype=np.int32),
                                n_features)[mask]
        data = np.ones(n_samples * n_features)[mask]

        out = sparse.csc_matrix((data, (row_indices, column_indices)),
                                shape=(n_samples, indices[-1]),
                                dtype=self.dtype).tocsr()
        if self.encoding == 'onehot-dense':
            return out.toarray()
        else:
            return out

In [30]:
# Retrieve the the target and feature values

In [0]:
X = train_data.iloc[:,:-1]
y = train_data.iloc[:,-1:]

In [31]:
# Split the the train data into train and test sets

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=28)

In [32]:
# The specifies the categorical versus numerical columns for data pre-processing

In [0]:
cat_attribs=["x34","x35","x68","x93"]
num_attribs=[e for e in list(X.columns) if e not in cat_attribs]

In [33]:
# Create a pipeline for data pre-processing

In [146]:
num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', Imputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])
cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_attribs)),
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ohe', OneHotEncoder(sparse=False, handle_unknown="ignore"))
    ])
full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [34]:
# Tranform the data

In [0]:
x_train = full_pipeline.fit_transform(X_train)
x_test = full_pipeline.fit_transform(X_test)

In [36]:
# The train and test sets above have been transformed and encoder and ready for training
# The first algorithm will be the XGBClassifier

#  XGBClassifier

In [0]:
score = pd.DataFrame(columns=['Classifier','AUC','Accuracy'])

In [0]:
xgtrain = xgb.XGBClassifier(max_depth=6,
                           min_child_weight=1,
                           learning_rate=0.1,
                           n_estimators=500,
                           silent=True,
                           objective='binary:logistic',
                           gamma=0.5,
                           max_delta_step=1,
                           subsample=0.5,
                           colsample_bytree=0.5,
                           colsample_bylevel=1,
                           reg_alpha=0,
                           reg_lambda=0,
                           scale_pos_weight=5,
                           seed=1,
                           random_state=28,
                           missing=None)

In [272]:
xgtrain.fit(x_train,y_train.values.ravel(),eval_metric='auc',verbose=False,eval_set=[(x_test,y_test)],early_stopping_rounds=100)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0.5, learning_rate=0.1,
       max_delta_step=1, max_depth=6, min_child_weight=1, missing=None,
       n_estimators=500, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=28, reg_alpha=0,
       reg_lambda=0, scale_pos_weight=5, seed=1, silent=True,
       subsample=0.5)

In [37]:
# Use the fitted algorithm to predict the sample test data and store the result in a dataframe

In [0]:
preds = xgtrain.predict(x_test)
pred_p = xgtrain.predict_proba(x_test)[:, 1]

In [0]:
accuracy = np.round(accuracy_score(y_test, preds),2)
auc = np.round(roc_auc_score(y_test,pred_p),2)

In [0]:
score.loc[len(score)] = ["XGBoostClassifier",auc,accuracy]

In [468]:
score

,Classifier,AUC,Accuracy
0,XGBoostClassifier,0.99,0.98


In [ ]:
# The classifier gives a AUC value of 0.99 and Accuracy of 0.98 on the sample test data.
# This is pretty good.

In [ ]:
# The second classifier will be the KNeighbors classifier.

# KNeighbors Classifier

In [0]:
from sklearn.neighbors import KNeighborsClassifier

In [0]:
neigh = KNeighborsClassifier(n_neighbors=3,
                             n_jobs=-1,
                             metric='euclidean',
                             weights='distance')

In [454]:
neigh.fit(x_train,y_train.values.ravel())

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=-1, n_neighbors=3, p=2,
           weights='distance')

In [38]:
# The traint Kneighbors classifier is the used to predict the sample test data abd the score stored in a dataframe

In [0]:
k_pred = neigh.predict(x_test)
k_pred_p = neigh.predict_proba(x_test)[:,1]

In [0]:
k_accuracy = np.round(accuracy_score(y_test, k_pred),2)
k_auc = np.round(roc_auc_score(y_test,k_pred_p),2)

In [0]:
score.loc[len(score)] = ["KNeighborsClassifier",k_auc,k_accuracy]

In [471]:
score

,Classifier,AUC,Accuracy
0,XGBoostClassifier,0.99,0.98
1,KNeighborsClassifier,0.93,0.92


In [ ]:
# The performance of the XGBClassifier is the better performer of the two algorithms.

In [39]:
# We can then used the algorithms to predict the y values of the provided test set

# Predict using Given Test Data

In [0]:
# First perform pre-processing and transformation of the given data.
x_test_val = full_pipeline.fit_transform(test_data)

In [484]:
x_test_val.shape

(10000, 127)

In [0]:
#Predit using XGBoostClassifier

In [0]:
preds_val1 = xgtrain.predict(x_test_val)

In [0]:
# Store the predicted results in a file
pd.DataFrame(preds_val1).to_csv('results1.csv',header=False,index=False)

In [0]:
#Predict using KNeighborsClassifier

In [0]:
preds_val2 = neigh.predict(x_test_val)

In [41]:
# Store the predicted results in a file
pd.DataFrame(preds_val2).to_csv('results2.csv',header=False,index=False)

The two given algorithms function pretty well. However, further model tuning like using gridsearch and randomsearch can be used to further improve performance. KNeighbors Classifier can be quite computationaly intensive due to the need to compute the distance between the samples. It is however quite robust especially when it comes to noisy data. XGboost has great execution speed and great performance.